# Cost of Living Index by Country

his report presents an analysis of cost of living indices across countries. A Random Forest classification model is employed to predict cost of living categories (Low, Medium, High). To address class imbalance, the Synthetic Minority Over-sampling Technique (SMOTE) is applied. Model performance is optimized through hyperparameter tuning.

## Import libraries

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imPipeline

## Load, understand, and analyze the data

Dataset URL: "https://raw.githubusercontent.com/cogxen/datasets/main/cost-of-living-index-by-country/cost-of-living-2024.csv"

- Load the dataset
- Assign the dataset to a variable

In [3]:
costs_url = "https://raw.githubusercontent.com/cogxen/datasets/main/cost-of-living-index-by-country/cost-of-living-2024.csv"

# Assign the dataset
costs = pd.read_csv(costs_url)

- Display the first five (5) rows of the DataFrame

In [4]:
costs.head()

,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,1,Switzerland,101.1,46.5,74.9,109.1,97.0,158.7
1,2,Bahamas,85.0,36.7,61.8,81.6,83.3,54.6
2,3,Iceland,83.0,39.2,62.0,88.4,86.8,120.3
3,4,Singapore,76.7,67.2,72.1,74.6,50.4,111.1
4,5,Barbados,76.6,19.0,48.9,80.8,69.4,43.5
